In [1]:
import numpy as np
from numpy import genfromtxt
from itertools import combinations
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
data = genfromtxt('small_file.txt', delimiter=',')
data[:,4] =(data[:,4]-np.min(data[:,4]))/(np.max(data[:,4])-np.min(data[:,4]))
print(data)

[[ 0.          1.128016    2.699852   -2.22828     0.27121927]
 [ 1.          0.952915    0.878809    0.519144    0.95261156]
 [ 1.          0.434082    2.311936   -2.573923    0.93668561]
 [ 2.          2.209619    2.334585    1.107064    0.52416023]
 [ 3.          1.71197     1.119164    1.189179    0.78435412]
 [ 3.          0.461599    0.144536   -2.364426    0.75403444]
 [ 4.          2.029792    1.303054    1.287916    0.        ]
 [ 5.          0.756178    2.309371   -1.809833    0.68407621]
 [ 6.          2.1872      2.85122     0.476108    0.8842718 ]
 [ 7.          1.921325    2.40708    -1.854133    1.        ]
 [ 7.          0.24677     1.512456    0.972258    0.95922116]
 [ 8.          2.12424     0.532672    1.190137    0.63181968]
 [ 9.          2.165567    1.332818    0.780826    0.52338727]
 [10.          1.927867    1.392146   -2.161047    0.83159162]
 [10.          0.155214    2.217043   -2.08358     0.81032574]
 [11.          2.17151     0.050683   -2.559912    0.75

### reshape data

In [3]:
#print(np.min(data,axis=0)[0])
ll = []
true_vals = []
for i in range(int(np.min(data,axis=0)[0]),int(np.max(data,axis=0)[0]),3):
    mask = ((data[:,0] == i) | (data[:,0] == i+1) | (data[:,0] == i+2))
    evt = data[mask]
    #print(evt)
    #print(evt.shape[0])
    pair_indices = list(combinations(range(evt.shape[0]), 2))
    #print(pair_indices)
    for i, (idx1, idx2) in enumerate(pair_indices):
        l = []
        l.append(np.concatenate((evt[idx1,1:],evt[idx2,1:])))
        ll.append(l)
        if (evt[idx1,0] == evt[idx2,0]):
            true_vals.append(1)
        else:
            true_vals.append(0)
input = torch.FloatTensor(ll)
input = torch.squeeze(input)
input.shape
truth_class = torch.FloatTensor(true_vals)
print(truth_class.shape)

torch.Size([60])


/tmp/ipykernel_20923/3555023645.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  input = torch.FloatTensor(ll)


### simple model

In [4]:
class feed_forward_model(nn.Module):
        def __init__(self):
                super().__init__()
                self.linear = torch.nn.Linear(8,10000)
                self.another_linear = torch.nn.Linear(10000,10000)
                self.activation = torch.nn.ReLU()
                self.linear_back = torch.nn.Linear(10000,1)

        def forward(self, x):
                output_tensor = self.linear(x)
                output_tensor = self.another_linear(output_tensor)
                output_tensor = self.activation(output_tensor)
                output_tensor = self.linear_back(output_tensor)
                output_tensor = torch.sigmoid(output_tensor)
                output_tensor = torch.squeeze(output_tensor)
                return output_tensor


## final training

In [ ]:
n_epochs = 10000
model = feed_forward_model()
loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.SGD(model.parameters(), lr=9e-4)
model.train()
loss_val = []
for epoch in range(n_epochs):
    y_pred = model(input)
    y_true = truth_class
    loss = loss_fn(y_pred,y_true)
    loss_val.append(loss.detach().item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 0.6474388241767883
Finished epoch 1, latest loss 0.52945476770401
Finished epoch 2, latest loss 0.4765293300151825
Finished epoch 3, latest loss 0.448923796415329
Finished epoch 4, latest loss 0.4323124587535858
Finished epoch 5, latest loss 0.42102691531181335
Finished epoch 6, latest loss 0.4125787615776062
Finished epoch 7, latest loss 0.40575864911079407
Finished epoch 8, latest loss 0.3999428153038025
Finished epoch 9, latest loss 0.3947921693325043
Finished epoch 10, latest loss 0.39010748267173767
Finished epoch 11, latest loss 0.3857714831829071
Finished epoch 12, latest loss 0.3817104697227478
Finished epoch 13, latest loss 0.3778763711452484
Finished epoch 14, latest loss 0.37423861026763916
Finished epoch 15, latest loss 0.37077176570892334
Finished epoch 16, latest loss 0.36745622754096985
Finished epoch 17, latest loss 0.3642788827419281
Finished epoch 18, latest loss 0.3612290918827057
Finished epoch 19, latest loss 0.3582965135574341
Finishe

In [ ]:
plt.plot(loss_val)
plt.xlabel("epochs")
plt.ylabel("loss value")